In [ ]:
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple akshare

In [ ]:
import pandas as pd
import akshare as ak
from datetime import datetime
import os

# 自动创建保存路径
def save_data(df: pd.DataFrame, variety_name: str, folder: str = "F:/国债期货数据/", filetype: str = "csv"):
    os.makedirs(folder, exist_ok=True)
    file_path = os.path.join(folder, f"{variety_name}_国债期货.{filetype}")
    if filetype == "csv":
        df.to_csv(file_path, index=False, encoding="utf-8-sig")
    elif filetype == "parquet":
        df.to_parquet(file_path, index=False)
    print(f"✅ 已保存 {variety_name} 数据，共 {len(df)} 条记录 -> {file_path}")

# 单月抓取
def fetch_futures_monthly(market: str, year: int, month: int):
    start = f"{year}{month:02d}01"
    if month == 12:
        end = f"{year+1}0101"
    else:
        end = f"{year}{month+1:02d}01"
    print(f"开始抓取：{start} 至 {end} ... ", end="")
    try:
        df = ak.get_futures_daily(start_date=start, end_date=end, market=market)
        print(f"✅ 成功（{len(df)} 条记录）")
        return df
    except Exception as e:
        print(f"❌ 失败，原因：{e}")
        return pd.DataFrame()

# 某品种历史数据抓取函数（按时间段）
def fetch_data_by_range(start_year: int, start_month: int, end_year: int, end_month: int, variety_code: str):
    data = []
    year = start_year
    month = start_month
    while (year < end_year) or (year == end_year and month <= end_month):
        print(f"[{datetime.now().strftime('%H:%M:%S')}] 抓取 {year} 年 {month} 月数据中 ...")
        df = fetch_futures_monthly("CFFEX", year, month)
        df = df[df["variety"] == variety_code]
        data.append(df)
        if month == 12:
            year += 1
            month = 1
        else:
            month += 1
    return pd.concat(data, ignore_index=True)

# 当前时间范围
current_year = datetime.now().year
current_month = datetime.now().month

# ===== 抓取并保存每个品种 =====

# TF（5年期）自2013年9月
tf_data = fetch_data_by_range(2013, 9, current_year, current_month, "TF")
save_data(tf_data, "TF_5年期")

# T（10年期）自2015年3月
t_data = fetch_data_by_range(2015, 3, current_year, current_month, "T")
save_data(t_data, "T_10年期")

# TS（2年期）自2018年8月
ts_data = fetch_data_by_range(2018, 8, current_year, current_month, "TS")
save_data(ts_data, "TS_2年期")

# TL（30年期）自2023年4月
tl_data = fetch_data_by_range(2023, 4, current_year, current_month, "TL")
save_data(tl_data, "TL_30年期")